In [ ]:
# ============ IMPORTS ============
from openai import OpenAI
import pandas as pd
import json
import os
import re

# ============ 1. API KEY & CLIENT SETUP ============
# Recommended: set via environment variable before running
# Example: export OPENAI_API_KEY="your_key_here"
os.environ["OPENAI_API_KEY"] = "your_api_keys_here"  # TODO: Replace before running

api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("ERROR: OPENAI_API_KEY environment variable not set.")

client = OpenAI(api_key=api_key)

# ============ 2. LOAD INPUT DATA ============
# Using Claude evidence file instead of GPT version
df = pd.read_excel("claude_safety_entries_eng.xlsx")

# ============ 3. DEFINE SCORING WEIGHTS ============
weights = {
    "menu_placement": 0.25,
    "iconography": 0.20,
    "label_clarity": 0.25,
    "user_effort": 0.15,
    "trust_confidence": 0.15
}

results = []  # To store final scoring outputs

# ============ 4. LOOP THROUGH EVIDENCE ROWS ============
for _, row in df.iterrows():
    prompt = f"""
You must score the following evidence excerpt based on these 5 criteria (scale 1–5):
1. Menu Placement Visibility
2. Iconography Prominence
3. Label Clarity
4. User Effort / Steps Required
5. Trust & UX Confidence

Use these weights:
- Menu Placement: 0.25
- Iconography: 0.20
- Label Clarity: 0.25
- User Effort: 0.15
- Trust & UX Confidence: 0.15

Final score = weighted sum.

Evidence:
- Source: {row['Evidence Source']}
- Excerpt: {row['Evidence Excerpt (English)']}
- Notes: {row['Measurement Notes (Manual Work)']}

⚠ OUTPUT REQUIREMENTS:
Return output STRICTLY as valid JSON only.
No markdown. No code block. No additional commentary.

Format must be:
{{
  "scores": {{
    "menu_placement": X,
    "iconography": X,
    "label_clarity": X,
    "user_effort": X,
    "trust_confidence": X
  }},
  "weighted_final_score": X,
  "justification": "short justification within 70 words"
}}
"""
    # GPT scoring execution
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are an ethics evaluation expert scoring safety entry prominence."
            },
            {"role": "user", "content": prompt}
        ]
    )

    raw_content = response.choices[0].message.content.strip()

    # ===== Robust JSON parsing =====
    try:
        result_json = json.loads(raw_content)
    except json.JSONDecodeError:
        match = re.search(r"\{[\s\S]*\}", raw_content)
        if match:
            try:
                result_json = json.loads(match.group())
            except json.JSONDecodeError:
                print("⚠ JSON could not be parsed even after extraction.\nRaw output:")
                print(raw_content)
                continue
        else:
            print("⚠ No JSON detected in response.\nRaw output:")
            print(raw_content)
            continue

    # Attach metadata
    result_json["Evidence ID"] = row["Evidence ID"]
    result_json["Evidence Source"] = row["Evidence Source"]

    # Retrieve raw scores
    s = result_json["scores"]

    # Compute weighted scores per dimension
    detailed_weighted_scores = {
        key: round(s[key] * weights[key], 2) for key in s
    }

    # Compute total final weighted score
    weighted_score = round(sum(detailed_weighted_scores.values()), 2)

    # Store structured scoring breakdown
    result_json["individual_scores_unweighted"] = s
    result_json["weighted_scores_detailed"] = detailed_weighted_scores
    result_json["weighted_final_score"] = weighted_score

    results.append(result_json)

# ============ 5. EXPORT RESULTS ============
# Save as new JSON file for Claude scoring
output_filename = "claude_scoring_weighted.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("🎉 Scoring successfully completed using GPT-4o.")
print(f"📁 Output saved: {output_filename}")


🎉 Scoring successfully completed using GPT-4o.
📁 Output saved: claude_scoring_weighted.json
